In [ ]:
import pandas as pd
import re
from glob import glob
from tqdm import tqdm
from os.path import dirname

In [ ]:
def get_columns(dirpath):

  filename = glob(dirpath+'/*.xlsx')[0]

  data = pd.read_excel(
        filename,
        sheet_name='report',
        engine='openpyxl',
        header=3,
        index_col=0,
        nrows=0)
  
  regex = re.compile(r'(\d\d\d\d), (.+)')

  common_cols = []
  mapping = {}

  for column_name in data.columns:
      feature = regex.match(column_name)

      if feature:
          year = int(feature.group(1))

          if year not in mapping:
            mapping[year] = {}
          mapping[year][column_name] = feature.group(2)
      else:
          common_cols.append(column_name)
  
  return common_cols, mapping



In [ ]:
def read_spark(dirpath):

  common_cols, mapping = get_columns(dirpath)
  
  result = None

  for filename in tqdm(glob(dirpath+'/*.xlsx'), desc="Files"):

    data = pd.read_excel(
        filename,
        sheet_name='report',
        engine='openpyxl',
        header=3,
        index_col=0)

    data.dropna(axis=0, subset=['Наименование'], inplace=True) #Удаление 2 пустых строчек в конце таблицы

    for year in mapping:
        df = data[common_cols + list(mapping[year].keys())]
        df = df.rename(columns=mapping[year])
        df.insert(9, 'Год', year, True)

        if result is not None:
            result = pd.concat([result, df])
        else:
            result = df

  result.reset_index(drop=True, inplace=True)
    
  return result

In [ ]:
dirpath = r'/content/drive/MyDrive/ЭкоТомск/practice3/СПАРК 2020'
df = read_spark(dirpath)
savepath = dirname(dirpath)
df.to_csv(f"{savepath}/spark.csv")
df.to_pickle(f"{savepath}/spark.pkl")


Files: 100%|██████████| 193/193 [27:58<00:00,  8.70s/it]
